In [1]:
import requests
import numpy as np
import datetime

Set up your API token from your profile on observation portal

In [2]:
API_TOKEN   = 'XXX'
PROPOSAL_ID = 'LCOEPO2019A-001' 
OBSERVATION_TITLE   = 'StudentData_Target_F'
TARGET_NAME         = 'TARGET_F'
RA                  = 42.54127
DEC                 = -26.26448
OBSERVATION_CADENCE = 24./60 # hours
START_TIME          = '2019-10-29 15:00:00' # UT 
END_TIME            = '2019-10-30 01:00:00' # UT
EXPOSURE_TIME       = 90 # seconds
FILTER1             = 'v'
FILTER2             = 'b'
print('The observation cadence is: ',OBSERVATION_CADENCE,' Hours, or ',OBSERVATION_CADENCE*60,' minutes')

The observation cadence is:  0.4  Hours, or  24.0  minutes


In [3]:
# Determine when the observations are to take place
# The period is NOT the period of the variable. 
# The period is the cadence at which you want to observe, in HOURS
# The Jitter is the window within which your observation may take place. 

# Want jitter to equal period to "fully cover" time
jitter = OBSERVATION_CADENCE

cadence = {
    'start': START_TIME,
    'end': END_TIME,
    'period': OBSERVATION_CADENCE,
    'jitter': jitter
}

In [4]:
target = {
    'name': TARGET_NAME,
    'type': 'ICRS',
    'ra': RA,
    'dec': DEC,
    'epoch': 2000
}

In [5]:
# Keep Max_airmass at 1.6
# min_lunar_distance is in DEGREES, keep at 20 degrees

constraints = {
    'max_airmass': 2.0,
    'min_lunar_distance': 20
}

In [6]:
# This does not need to be changed.
# Exposure time is in SECONDS

configurations = [
    {
        'type': 'EXPOSE',
        'instrument_type': '0M4-SCICAM-SBIG',
        'target': target,
        'constraints': constraints,
        'acquisition_config': {},
        'guiding_config': {},
        'instrument_configs': [
            {
                'exposure_time': EXPOSURE_TIME,
                'exposure_count': 1,
                'optical_elements': {
                    'filter': FILTER1
                }
            }
        ]
    },
    {
        'type': 'EXPOSE',
        'instrument_type': '0M4-SCICAM-SBIG',
        'target': target,
        'constraints': constraints,
        'acquisition_config': {},
        'guiding_config': {},
        'instrument_configs': [
            {
                'exposure_time': EXPOSURE_TIME,
                'exposure_count': 1,
                'optical_elements': {
                    'filter': FILTER2
                }
            }
        ]
    }
]

In [7]:
# This cell does not need to be edited.

# We do not provide windows for a cadence request
windows = []

# The telescope class that should be used for this observation
location = {
    'telescope_class': '0m4'
}

In [8]:
# Please input the name of your observation request. Eg. Group_Star_Target_B
requestgroup = {
    'name': OBSERVATION_TITLE,  # The title
    'proposal': PROPOSAL_ID,
    'ipp_value': 1.05,
    'operator': 'SINGLE',
    'observation_type': 'NORMAL',
    'requests': [{
        'cadence': cadence,
        'configurations': configurations,
        'windows': windows,
        'location': location,
    }]
}

In [9]:
response = requests.post(
    'https://observe.lco.global/api/requestgroups/cadence/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=requestgroup  # Make sure you use json!
)

In [10]:
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('Request failed: {}'.format(response.content))
    raise exc

In [11]:
cadence_request = response.json()

print('Cadence generated {} requests'.format(len(cadence_request['requests'])))
i = 1
for request in cadence_request['requests']:
    print('Request {0} window start: {1} window end: {2}'.format(
        i, request['windows'][0]['start'], request['windows'][0]['end']
    ))
    i = i + 1

Cadence generated 25 requests
Request 1 window start: 2019-10-29T15:00:00Z window end: 2019-10-29T15:12:00Z
Request 2 window start: 2019-10-29T15:12:00Z window end: 2019-10-29T15:36:00Z
Request 3 window start: 2019-10-29T15:36:00Z window end: 2019-10-29T16:00:00Z
Request 4 window start: 2019-10-29T16:00:00Z window end: 2019-10-29T16:24:00Z
Request 5 window start: 2019-10-29T16:24:00Z window end: 2019-10-29T16:48:00Z
Request 6 window start: 2019-10-29T16:48:00Z window end: 2019-10-29T17:12:00Z
Request 7 window start: 2019-10-29T17:12:00Z window end: 2019-10-29T17:36:00Z
Request 8 window start: 2019-10-29T17:36:00Z window end: 2019-10-29T18:00:00Z
Request 9 window start: 2019-10-29T18:00:00Z window end: 2019-10-29T18:24:00Z
Request 10 window start: 2019-10-29T18:24:00Z window end: 2019-10-29T18:48:00Z
Request 11 window start: 2019-10-29T18:48:00Z window end: 2019-10-29T19:12:00Z
Request 12 window start: 2019-10-29T19:12:00Z window end: 2019-10-29T19:36:00Z
Request 13 window start: 2019-1

In [12]:
# Submit the fully formed RequestGroup
response = requests.post(
    'https://observe.lco.global/api/requestgroups/validate/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=cadence_request  # Make sure you use json!
)

# Make sure the API call was successful
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('API call failed: {}'.format(response.content))
    raise exc

requestgroup_dict = response.json()  # The API will return the newly submitted requestgroup as json

print('Total number of observing hours needed for this request: ',
      requestgroup_dict['request_durations']['duration']/60./60)

Total number of observing hours needed for this request:  2.319444444444444


# Do not run the following cell until you have confirmed with an instructor your observation.

In [13]:
response = requests.post(
    'https://observe.lco.global/api/requestgroups/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=cadence_request  # Make sure you use json!
)

# Make sure this API call was successful
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('Request failed: {}'.format(response.content))
    raise exc

requestgroup_dict = response.json()

# Print out the url on the portal where we can view the submitted request
print('View this observing request: https://observe.lco.global/requestgroups/{}/'.format(requestgroup_dict['id']))

print('This request was successfully submitted at {}'.format(datetime.datetime.now()))

View this observing request: https://observe.lco.global/requestgroups/877592/
This request was successfully submitted at 2019-10-29 12:28:26.608760
